#### FUNCIONES

In [1]:
import pandas as pd

In [2]:
# lectura de archivos 
df_steam_games=pd.read_csv('steam_games.csv')
df_user_items=pd.read_csv('user_items.csv')
df_user_reviews=pd.read_csv('user_reviews.csv')

##### **Funcion 1** ______________________________________________________________________________________________________________________________________________________________________________

In [37]:
df_funcion1=pd.merge(df_user_items,df_steam_games,on='id_juego',how='left')

In [38]:
df_funcion1=df_funcion1.loc[:,['genero','año_lanzamiento','timpo_total_jugado']].groupby(by=['genero','año_lanzamiento']).sum()

In [39]:
df_funcion1.reset_index(inplace=True)

In [40]:
df_funcion1.dropna(inplace=True)

In [74]:
df_funcion1 # df definitivo para la funcion 1 

,genero,año_lanzamiento,timpo_total_jugado
0,Action,1983.0,3582
1,Action,1984.0,384
2,Action,1988.0,16243
3,Action,1989.0,607
4,Action,1990.0,18723
...,...,...,...
317,Web Publishing,2013.0,140060
318,Web Publishing,2014.0,22425
319,Web Publishing,2015.0,332496
320,Web Publishing,2016.0,1


In [69]:
df_funcion1.to_csv('funcion1.csv',index=False) # pasar a a csv, este archivo es el que voy a subir a la api

In [206]:
def PlayTimeGenre( genre : str ): #  Debe devolver año con mas horas jugadas para dicho género.
    # pasar a dataframe dentro de la funcion ?
    try:
        valor_maximo=df_funcion1[df_funcion1['genero']==genre]['timpo_total_jugado'].max()
        indice=df_funcion1[df_funcion1['timpo_total_jugado']==valor_maximo].index
        resultado=df_funcion1['año_lanzamiento'].loc[indice].values
        return {f'Año de lanzamiento con mas horas jugadas para el Género {genre}:':resultado[0]}
    except Exception as e:
        print('Genero incorrecto')


# Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

# Como unir las tablas ? 
    # usar codigo SQL
    # pandas con merge 
    # pandas con join 

In [209]:
PlayTimeGenre('Action')

{'Año de lanzamiento con mas horas jugadas para el Género Action:': 2012.0}

##### **Funcion 2** ______________________________________________________________________________________________________________________________________________________________________________

In [78]:
# elegir el dataframe reducido a utilizar

df_steam_games.head(3) # Exploracion 

,publisher,genero,title,fecha_lanzamiento,price,early_access,id_juego,developer,año_lanzamiento
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,4.99,0.0,761140,Kotoshiro,2018
1,Kotoshiro,Casual,Lost Summoner Kitty,2018-01-04,4.99,0.0,761140,Kotoshiro,2018
2,Kotoshiro,Indie,Lost Summoner Kitty,2018-01-04,4.99,0.0,761140,Kotoshiro,2018


In [79]:
df_user_items.head(3) # Exploracion

,user_id,id_juego,nombre_juego,timpo_total_jugado,tiempo_jugado_ultimas_2semanas
0,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,30,Day of Defeat,7,0


In [80]:
df_user_reviews.head(3) # Exploracion 

,user_id,funny,last_edited,id_juego,helpful,recommend,sentiment_analysis,review_posted,año_posted_review
0,76561197970982479,NaN,NaN,1250,No ratings yet,True,2,2011-11-05,2011
1,76561197970982479,NaN,NaN,43110,No ratings yet,True,2,2011-04-21,2011
2,js41637,NaN,NaN,251610,15 of 20 people (75%) found this review helpful,True,2,2014-06-24,2014


In [83]:
df_funcion2=pd.merge(df_user_items,df_steam_games,on='id_juego',how='left').merge(df_user_reviews,on=['user_id','id_juego'],how='left')

In [86]:
df_funcion2=df_funcion2.loc[:,['genero','user_id','año_posted_review','timpo_total_jugado']] # seleccionar columnas que necesito para la funcion

In [95]:
# Ver valores nulos de cada columna 
for columna in df_funcion2.columns:
    print(f'{columna} ',df_funcion2[columna].isnull().sum(),'nulos')

#   Desicion: dado que necesito obligatoriamente el dato de "genero" y "año", desido eliminar los nulos en las mencionadas columnas, se observa que para algunos juegos no existe ni genero ni año_posted_Review
# lo cual indica que hay jugadores que han jugado un juego pero no han hecho una review del mismo 

genero  1116153 nulos
user_id  0 nulos
año_posted_review  10636404 nulos
timpo_total_jugado  0 nulos


In [98]:
# elimino nulos 
df_funcion2.dropna(inplace=True)

In [100]:
df_funcion2.groupby(by=['genero','user_id','año_posted_review']).sum().reset_index(inplace=True)

In [101]:
df_funcion2  # df definitivo 

,genero,user_id,año_posted_review,timpo_total_jugado
55,Action,76561197970982479,2011.0,10006
1003,Indie,js41637,2013.0,551
1004,Simulation,js41637,2013.0,551
1175,Adventure,js41637,2013.0,349
1176,Indie,js41637,2013.0,349
...,...,...,...,...
10709491,Action,76561198239215706,2015.0,4659
10710333,Action,wayfeng,2015.0,42740
10710470,RPG,76561198251004808,2015.0,1098
10711601,Action,72947282842,2015.0,33


In [103]:
df_funcion2.to_csv('funcion2.csv',index=False)

In [210]:
df_funcion2

,genero,user_id,año_posted_review,timpo_total_jugado
55,Action,76561197970982479,2011.0,10006
1003,Indie,js41637,2013.0,551
1004,Simulation,js41637,2013.0,551
1175,Adventure,js41637,2013.0,349
1176,Indie,js41637,2013.0,349
...,...,...,...,...
10709491,Action,76561198239215706,2015.0,4659
10710333,Action,wayfeng,2015.0,42740
10710470,RPG,76561198251004808,2015.0,1098
10711601,Action,72947282842,2015.0,33


In [50]:
def UserForGenre( genero : str ): #Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
    pass

# Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

##### **Funcion 3** ______________________________________________________________________________________________________________________________________________________________________________

In [111]:
df_funcion3=pd.merge(df_user_reviews,df_user_items,on=['user_id','id_juego'],how='left').merge(df_steam_games,on='id_juego',how='left')

In [112]:
df_funcion3.head(3)

,user_id,funny,last_edited,id_juego,helpful,recommend,sentiment_analysis,review_posted,año_posted_review,nombre_juego,timpo_total_jugado,tiempo_jugado_ultimas_2semanas,publisher,genero,title,fecha_lanzamiento,price,early_access,developer,año_lanzamiento
0,76561197970982479,NaN,NaN,1250,No ratings yet,True,2,2011-11-05,2011,Killing Floor,10006.0,0.0,Tripwire Interactive,Action,Killing Floor,2009-05-14,19.99,0.0,Tripwire Interactive,2009.0
1,76561197970982479,NaN,NaN,43110,No ratings yet,True,2,2011-04-21,2011,Metro 2033,834.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,js41637,NaN,NaN,251610,15 of 20 people (75%) found this review helpful,True,2,2014-06-24,2014,Barbie™ Dreamhouse Party™,84.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df_funcion3=df_funcion3[(df_funcion3['recommend']==True) & ((df_funcion3['sentiment_analysis']==2) | (df_funcion3['sentiment_analysis']==1))]

In [114]:
df_funcion3=df_funcion3.loc[:,['id_juego','title','año_posted_review','recommend']]

In [115]:
df_funcion3

,id_juego,title,año_posted_review,recommend
0,1250,Killing Floor,2011,True
1,43110,NaN,2011,True
2,251610,NaN,2014,True
7,370360,TIS-100,2015,True
8,370360,TIS-100,2015,True
...,...,...,...,...
108893,730,Counter-Strike: Global Offensive,2015,True
108894,730,Counter-Strike: Global Offensive,2015,True
108895,253980,Enclave,2015,True
108897,730,Counter-Strike: Global Offensive,2015,True


In [116]:
df_funcion3=df_funcion3.groupby(by=['id_juego','title','año_posted_review']).count()

In [117]:
df_funcion3.reset_index(inplace=True)

In [118]:
df_funcion3

,id_juego,title,año_posted_review,recommend
0,10,Counter-Strike,2011,1
1,10,Counter-Strike,2012,3
2,10,Counter-Strike,2013,7
3,10,Counter-Strike,2014,18
4,10,Counter-Strike,2015,15
...,...,...,...,...
3314,419070,Warhammer: End Times - Vermintide Sigmar's Ble...,2015,6
3315,421630,A Study in Steampunk: Choice by Gaslight,2015,2
3316,423120,Community College Hero: Trial by Fire,2015,2
3317,423880,Carpe Diem,2015,27


In [119]:
df_funcion3=df_funcion3.loc[:,['año_posted_review','id_juego','title','recommend']]  # reordear data set a fines de mejor lectura 

In [121]:
df_funcion3.sort_values(by='año_posted_review',ascending=True)  # EXPLORACION , trabajar con este dataset, tomarlo como definitivo

,año_posted_review,id_juego,title,recommend
429,2010,13570,Tom Clancy's Splinter Cell Chaos Theory®,1
810,2010,39690,ArcaniA,1
620,2010,24010,Train Simulator,1
608,2010,22600,Worms Reloaded,2
32,2010,240,Counter-Strike: Source,2
...,...,...,...,...
1888,2015,233450,Prison Architect,108
1883,2015,233290,Murdered: Soul Suspect,6
1881,2015,233270,Far Cry 3 - Blood Dragon,2
1875,2015,233130,Shadow Warrior,12


In [123]:
# Exportar dataframe a csv con el que voy a trabajar definitivamente la funcion 3

df_funcion3.to_csv('funcion3.csv',index=False)

In [61]:
def UsersRecommend( año : int ): # Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
    pass

#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

##### **Funcion 4** ______________________________________________________________________________________________________________________________________________________________________________

In [125]:
df_funcion4=pd.merge(df_user_reviews,df_user_items,on=['user_id','id_juego'],how='left').merge(df_steam_games,on='id_juego',how='left')

In [126]:
df_funcion4=df_funcion4[(df_funcion4['recommend']==False) & (df_funcion4['sentiment_analysis']==0)]

In [129]:
df_funcion4=df_funcion4.loc[:,['año_posted_review','id_juego','title','recommend']]

In [132]:
df_funcion4=df_funcion4.groupby(by=['año_posted_review','id_juego','title']).count()

In [134]:
df_funcion4.reset_index(inplace=True)

In [135]:
df_funcion4  # data listao para hacer la funcion , deberia eliminar los nulos y revisar un poco 

,año_posted_review,id_juego,title,recommend
0,2011,440,Team Fortress 2,2
1,2011,18700,And Yet It Moves,4
2,2011,33460,From Dust,2
3,2011,63940,Men of War: Vietnam,1
4,2012,440,Team Fortress 2,2
...,...,...,...,...
815,2015,409510,Genesis Online,2
816,2015,410340,Liftoff,5
817,2015,412400,GASP,6
818,2015,417860,Emily is Away,20


In [137]:
df_funcion4.to_csv('funcion4.csv',index=False) # pasar a csv 

In [67]:
def UsersNotRecommend( año : int ): # Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
    pass

# Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

##### **Funcion 5** ______________________________________________________________________________________________________________________________________________________________________________

In [68]:
def sentiment_analysis( año : int ): # Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
    pass

# Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}